<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <center>
    Click into a code cell (the gray blocks below) to select or edit it.<br/>
    To run a selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    Make sure that each code cell runs successfully before you move on to the next one.
    </center>
</div>

# Simple advection-diffusion model

This notebook implements a simple 2D advection-diffusion model using Landlab, with a prescribed uniform advection velocity.

*(GT, inspired by a question from Shemin Ge, Oct 2020)*


Imports:

In [ ]:
import numpy as np
from landlab import RasterModelGrid, imshow_grid
from landlab.grid.mappers import map_link_tail_node_to_link

In [ ]:
# Parameters
nrows = 11
ncols = 11
dx = 1.0 # grid spacing, m
diffusivity = 2.0e-4 # m2/s
velocity = 0.001 # m/s
initial_concentration = 1.0 # volume concentration of solute
grid_node_with_starting_solute = ncols * (nrows//2) + 1 #ncols * (nrows//2) + nrows//2
run_duration = 5000.0

# Derived parameters
dt = 0.2 * dx * dx / diffusivity
dt_advec = 0.5 * dx / velocity
dt = min(dt, dt_advec)
nsteps = int(round(run_duration / dt))

In [ ]:
grid = RasterModelGrid((nrows, ncols), xy_spacing=dx)
conc = grid.add_zeros('solute_concentration', at='node')
vel = grid.add_zeros('flow_velocity', at='link')
flux = grid.add_zeros('solute_flux', at='link')
conc[grid_node_with_starting_solute] = initial_concentration
vel[grid.horizontal_links] = velocity

In [ ]:
# initial concentration field
imshow_grid(grid, conc)

In [ ]:
for i in range(nsteps):
    conc_grad = grid.calc_grad_at_link(conc)
    diff_flux = -diffusivity * conc_grad  # Fick's Law
    conc_at_links = map_link_tail_node_to_link(grid,
                                               'solute_concentration')
    advec_flux = vel * conc_at_links
    total_flux = diff_flux + advec_flux
    dcdt = -grid.calc_flux_div_at_node(total_flux)
    conc[:] += dcdt * dt

In [ ]:
# final concentration field
imshow_grid(grid, conc)